# Импорт библиотек

In [5]:
# настройка ширины страницы блокнота .......................................
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# расширение watermark для вывода информации о версиях пакетов
# https://github.com/rasbt/watermark
# %load_ext watermark

In [6]:
# загрузка пакетов: инструменты --------------------------------------------
# работа с массивами
import numpy as np
# фреймы данных
import pandas as pd
# графики
import matplotlib as mpl
# стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# загрузка файлов по URL
import urllib
# проверка существования файла на диске
from pathlib import Path
# для форматирования результатов с помощью Markdown
from IPython.display import Markdown, display
# перекодировка категориальных переменных
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# хи-квадрат тест на независимость по таблице сопряжённости
from scipy.stats import chi2_contingency
# для таймера
import time
# загрузка пакетов: данные -------------------------------------------------
from sklearn import datasets
# загрузка пакетов: модели -------------------------------------------------
# дерево классификации
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# бэггинг
from sklearn.ensemble import BaggingClassifier
# случайный лес
from sklearn.ensemble import RandomForestClassifier
# бустинг
from sklearn.ensemble import GradientBoostingClassifier
# сводка по точности классификации
from sklearn.metrics import classification_report

In [7]:
import datetime
import os
from collections import Counter

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
    
from sklearn.decomposition import NMF, KernelPCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, RandomizedSearchCV, \
    HalvingGridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.svm import SVC

import numpy as np
from scipy import stats

In [8]:
# константы
# ядро для генератора случайных чисел
my_seed = 17
# создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
# примеры стилей и шаблонов графиков:
# http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-v0_8')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

# Загрузка данных

In [9]:
localDataPath = './data/'
localFileName = 'heart.csv'
localFilePath = os.path.join(localDataPath, localFileName)

!kaggle datasets download -f {localFileName} -p {localDataPath} --unzip fedesoriano/heart-failure-prediction

heart.csv: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
DF_raw = pd.read_csv(localFilePath)
DF_raw

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [11]:
DF_raw.dtypes

Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

In [12]:
DF_raw.iloc[:, :6].head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS
0,40,M,ATA,140,289,0
1,49,F,NAP,160,180,0
2,37,M,ATA,130,283,0
3,48,F,ASY,138,214,0
4,54,M,NAP,150,195,0


In [13]:
DF_raw.iloc[:, 6:12].head()

,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,Normal,172,N,0.0,Up,0
1,Normal,156,N,1.0,Flat,1
2,ST,98,N,0.0,Up,0
3,Normal,108,Y,1.5,Flat,1
4,Normal,122,N,0.0,Up,0


Разделим признаки на числовые, номинальные и порядковые.

Признак `FastingBS` уже закодирован бинарно и не требует преобразования, поэтому отнесем его к числовым.

In [14]:
num_col_names = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak',
                 'FastingBS']
ord_col_names = ['ST_Slope']
nom_col_names = ['Sex', 'RestingECG', 'ChestPainType', 'ExerciseAngina']

# проверка на опечатки
def sanity_check_col_list(col_list, df):
    for col in col_list:
        assert col in df.columns

sanity_check_col_list(num_col_names + ord_col_names + nom_col_names,
                      DF_raw)

Проверим на наличие пропущенных значений

In [15]:
DF_raw.isna().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

Проверим доли классов

In [16]:
def display_freq(col):
    col_freq = np.round(DF_raw[col].value_counts(dropna=False) / DF_raw[col].shape[0], 2)
    display(col_freq.sort_index())
    print('Всего уникальных значений: ', col_freq.shape[0])

In [17]:
display_freq('HeartDisease')

HeartDisease
0    0.45
1    0.55
Name: count, dtype: float64

Всего уникальных значений:  2


Отложим 15% наблюдений для прогноза

In [18]:
DF_num = DF_raw.sample(frac=0.85, random_state=my_seed)
DF_predict_num = DF_raw.drop(DF_num.index)

## Преобразование данных и построение моделей

Определим пайплайны предобработки данных.

*(Замена пустых значений с помощью различных `Imputers` приведена для демонстрации - в наших данных пустых значений нет)*

In [19]:
# help transformer
class DenseTransformer(TransformerMixin):
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        X = np.asarray(X.todense())
        return X

In [20]:
num_preproc = Pipeline(
    steps=[
        ('imputation_knn', KNNImputer(missing_values=np.nan)),
    ]
)

enc_ST_Slope = ['Down', 'Flat', 'Up']
ord_preproc = Pipeline(
    steps=[
        ('encoder_ord', OrdinalEncoder(categories=[enc_ST_Slope],
                                       unknown_value=np.nan,
                                       handle_unknown='use_encoded_value',
                                       encoded_missing_value=np.nan)),
        ('imputation_median', SimpleImputer(strategy='median'))
    ]
)


nom_preproc = Pipeline(
    steps=[
        ('encoder_onehot', OneHotEncoder(handle_unknown='ignore')),
        ('to_dense', DenseTransformer()),
        ('imputation_RFC', IterativeImputer(estimator=RandomForestClassifier(),
                                            initial_strategy='most_frequent',
                                            max_iter=10, random_state=my_seed))
    ]
)

preprocessor = ColumnTransformer(
    [
        ('numerical', num_preproc, num_col_names),
        ('ordinal', ord_preproc, ord_col_names),
        ('nominal', nom_preproc, nom_col_names)
    ]
)


## Построение SVC

Разделим подбор гиперпараметров для SVC на два этапа, чтобы уменьшить размерность пространство перебора и сократить время выполнения. Сначала подберем лучший метод снижения размерности, а затем подберем гиперпараметры самого классификатора

### Tuning снижения размерности

In [21]:
X_train = DF_num.drop(columns=['HeartDisease'])
y_train = DF_num['HeartDisease']

In [24]:
pipe_reduce_dim = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('scaler', StandardScaler()),
        ('reduce_dim', 'passthrough'),
        ('classifier', SVC())
    ]
)
pipe_reduce_dim

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputation_knn',
                                                                   KNNImputer())]),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak', 'FastingBS']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('encoder_ord',
                                                                   OrdinalEncoder(categories=[['Down',
                                                                                               'Flat',
                                                                                               'Up']],
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan)),
                                                                  ('imputation...
                                                                   OneHotEncoder(handle_unknown='ignore')),
                                                                  ('to_dense',
                                                                   <__main__.DenseTransformer object at 0x000002EC5A652CE0>),
                                                                  ('imputation_RFC',
                                                                   IterativeImputer(estimator=RandomForestClassifier(),
                                                                                    initial_strategy='most_frequent',
                                                                                    random_state=17))]),
                                                  ['Sex', 'RestingECG',
                                                   'ChestPainType',
                                                   'ExerciseAngina'])])),
                ('scaler', StandardScaler()), ('reduce_dim', 'passthrough'),
                ('classifier', SVC())])

In [25]:
N_COMPONENTS_OPTIONS = [ 2, 4, 6 ]
param_grid_reduce_dim = [
    {
        'reduce_dim': [KernelPCA()],
        'reduce_dim__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'],
        'reduce_dim__n_components': N_COMPONENTS_OPTIONS
    },
    {
        'reduce_dim': [SelectKBest(mutual_info_classif)],
        'reduce_dim__k': N_COMPONENTS_OPTIONS
    }
]

search_reduce_dim = HalvingGridSearchCV(
    pipe_reduce_dim,
    random_state=my_seed,
    param_grid=param_grid_reduce_dim,
    scoring='accuracy',
    error_score='raise',
    factor=3
)

search_reduce_dim

HalvingGridSearchCV(error_score='raise',
                    estimator=Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('numerical',
                                                                                Pipeline(steps=[('imputation_knn',
                                                                                                 KNNImputer())]),
                                                                                ['Age',
                                                                                 'RestingBP',
                                                                                 'Cholesterol',
                                                                                 'MaxHR',
                                                                                 'Oldpeak',
                                                                                 'FastingBS']),
                                                                               ('ordinal',
                                                                                Pipeline(steps=[('encoder_ord',
                                                                                                 OrdinalEncoder(categories=[['Down',
                                                                                                                             'Flat',
                                                                                                                             'Up']],
                                                                                                                handle_unknown='u...
                                              ('scaler', StandardScaler()),
                                              ('reduce_dim', 'passthrough'),
                                              ('classifier', SVC())]),
                    param_grid=[{'reduce_dim': [KernelPCA()],
                                 'reduce_dim__kernel': ['linear', 'poly', 'rbf',
                                                        'sigmoid', 'cosine'],
                                 'reduce_dim__n_components': [2, 4, 6]},
                                {'reduce_dim': [SelectKBest(score_func=<function mutual_info_classif at 0x000002EC5A639480>)],
                                 'reduce_dim__k': [2, 4, 6]}],
                    random_state=17, scoring='accuracy')

In [26]:
tic = time.perf_counter()
search_pipe_reduce_dim = search_reduce_dim.fit(X_train, y_train)
toc = time.perf_counter()

print(f"Time elapsed: {datetime.timedelta(seconds=toc-tic)}")

Time elapsed: 0:03:41.427556


In [27]:
print(f'Best score: {search_pipe_reduce_dim.best_score_:.2f}')
print(f'Best params: {search_pipe_reduce_dim.best_params_}')

Best score: 0.84
Best params: {'reduce_dim': SelectKBest(score_func=<function mutual_info_classif at 0x000002EC5A639480>), 'reduce_dim__k': 6}


### Tuning классификатора SVC

Оптимизируемые гиперпараметры **SVC**:
* ядерная функция $kernel$
* параметр регуляризации $C$
* коэффициент ядерной функции $\gamma$

Для снижения размерности выберем лучший метод с предыдущего шага.

In [28]:
best_reduce_dim = search_pipe_reduce_dim.best_estimator_['reduce_dim']

pipe_svc = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('scaler', StandardScaler()),
        ('reduce_dim', best_reduce_dim),
        ('clf', SVC())
    ]
)
pipe_svc

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputation_knn',
                                                                   KNNImputer())]),
                                                  ['Age', 'RestingBP',
                                                   'Cholesterol', 'MaxHR',
                                                   'Oldpeak', 'FastingBS']),
                                                 ('ordinal',
                                                  Pipeline(steps=[('encoder_ord',
                                                                   OrdinalEncoder(categories=[['Down',
                                                                                               'Flat',
                                                                                               'Up']],
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=nan)),
                                                                  ('imputation...
                                                                   <__main__.DenseTransformer object at 0x000002EC5A652CE0>),
                                                                  ('imputation_RFC',
                                                                   IterativeImputer(estimator=RandomForestClassifier(),
                                                                                    initial_strategy='most_frequent',
                                                                                    random_state=17))]),
                                                  ['Sex', 'RestingECG',
                                                   'ChestPainType',
                                                   'ExerciseAngina'])])),
                ('scaler', StandardScaler()),
                ('reduce_dim',
                 SelectKBest(k=6,
                             score_func=<function mutual_info_classif at 0x000002EC5A639480>)),
                ('clf', SVC())])

In [56]:
param_range = stats.uniform(1e-4, 1e+4)
param_distr_svc = [
    {
        'clf__kernel': ['linear', 'rbf'],
        'clf__gamma': param_range,
        'clf__C': param_range,
    },
    {
        'clf__kernel': ['poly'],
        'clf__gamma': param_range,
        'clf__C': param_range,
        'clf__degree': [2, 3]
    }
]

strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=my_seed)
search_classifier = HalvingRandomSearchCV(
    pipe_svc,
    param_distributions=param_distr_svc,
    cv=strat_kfold,
    scoring='accuracy',
    error_score='raise',
    random_state=my_seed,
    factor=3,
    n_jobs=-1,
    n_candidates=9
)

In [57]:
tic = time.perf_counter()
search_pipe_svc = search_classifier.fit(X_train, y_train)
toc = time.perf_counter()

print(f"Time elapsed: {datetime.timedelta(seconds=toc-tic)}")

Time elapsed: 0:01:12.003475


In [58]:
print(f'Best score: {search_pipe_svc.best_score_:.2f}')
print(f'Best params: {search_pipe_svc.best_params_}')

Best score: 0.81
Best params: {'clf__C': 3778.5406876478673, 'clf__gamma': 11.239099495335466, 'clf__kernel': 'linear'}


## Построение Random Forest

Выберем следующие гиперпараметры **Random Forest** для оптимизации:
* количество деревьев $B$
* количество признаков для построения отдельного дерева $m$


In [60]:
# RF param ranges
max_features_range = ['log2', 'sqrt']
n_estimators_range = stats.randint(20, 60)
pipe_rf = Pipeline(
    steps=[
        ('preproc', preprocessor),
        ('scaler', StandardScaler()),
        ('clf', RandomForestClassifier())
    ]
)

param_distr_rf = {
    'clf__n_estimators': n_estimators_range,
    'clf__max_features': max_features_range,
}

strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=my_seed)
search_rf = HalvingRandomSearchCV(
    estimator=pipe_rf,
    param_distributions=param_distr_rf,
    cv=strat_kfold,
    scoring='accuracy',
    factor=3,
    n_jobs=-1,
    n_candidates=9
)

In [61]:
tic = time.perf_counter()
best_rf = search_rf.fit(X_train, y_train)
toc = time.perf_counter()

print(f"Time elapsed: {datetime.timedelta(seconds=toc-tic)}")

Time elapsed: 0:00:28.673582


In [73]:
print(f'Best score: {best_rf.best_score_:.2f}')
print(f'Best params: {best_rf.best_params_}')

Best score: 0.87
Best params: {'clf__max_features': 'sqrt', 'clf__n_estimators': 41}


# Прогноз на отложенные наблюдения

In [75]:
X_pred = DF_predict_num.drop(['HeartDisease'], axis=1)
y_hat = best_rf.best_estimator_.predict(X_pred)
print(classification_report(DF_predict_num['HeartDisease'], y_hat))

              precision    recall  f1-score   support

           0       0.84      0.83      0.83        58
           1       0.88      0.89      0.88        80

    accuracy                           0.86       138
   macro avg       0.86      0.86      0.86       138
weighted avg       0.86      0.86      0.86       138

